In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
from tqdm import tqdm
import re
import sys


# 바코드연계제품정보 데이터

In [7]:
# 바코드연계제품정보 (brcd1)
data_path_1 = os.path.join(os.pardir, 'data','바코드연계제품정보_20220529.csv')
brcd1 = pd.read_csv(data_path_1, dtype=str)

../data/바코드연계제품정보_20220529.csv


In [3]:
brcd1.head(3)

,CLSBIZ_DT,SITE_ADDR,PRDLST_REPORT_NO,PRMS_DT,PRDLST_NM,BAR_CD,POG_DAYCNT,PRDLST_DCNM,BSSH_NM,END_DT,INDUTY_NM
0,NaN,충청북도 진천군 이월면 궁동길 51-24,19930443028350,20070119,면사랑 구수한 메밀국수,8802039211424,24개월 실온,건면,(주)면사랑,NaN,식품제조가공업
1,NaN,경기도 광주시 독고개길 156,1987036804531,20000110,종합캔디,8801696003397,제조일로부터18개월,캔디류,(주)한일제과,NaN,식품제조가공업
2,NaN,부산광역시 동구 충장대로 293(좌천동),19950129282236,20090724,좋은상품 유기농 부침가루,8805522061018,제조일로부터 12개월까지,기타가공품,사조동아원(주) 부산공장,20200217,식품제조가공업


In [13]:
brcd1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244549 entries, 0 to 244548
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   CLSBIZ_DT         30129 non-null   object
 1   SITE_ADDR         244533 non-null  object
 2   PRDLST_REPORT_NO  244549 non-null  object
 3   PRMS_DT           244549 non-null  object
 4   PRDLST_NM         244511 non-null  object
 5   BAR_CD            244549 non-null  object
 6   POG_DAYCNT        238210 non-null  object
 7   PRDLST_DCNM       244295 non-null  object
 8   BSSH_NM           244524 non-null  object
 9   END_DT            69984 non-null   object
 10  INDUTY_NM         244545 non-null  object
dtypes: object(11)
memory usage: 20.5+ MB


### 1. 생산 중단일 “END_DT”  datetime 변환, 생산 중단 된지 1년 이상된 제품 제거

In [14]:
# 생산 중단일 타입 변환
brcd1['END_DT'] = pd.to_datetime(brcd1['END_DT'])
# 생산 중단일 1년 이상인 제품 제거
con_r = brcd1[brcd1['END_DT'] < (datetime.today() - timedelta(days=365))].index
brcd1.drop(con_r, inplace=True)
brcd1.reset_index(inplace=True, drop=True)

In [15]:
brcd1.head(3)

,CLSBIZ_DT,SITE_ADDR,PRDLST_REPORT_NO,PRMS_DT,PRDLST_NM,BAR_CD,POG_DAYCNT,PRDLST_DCNM,BSSH_NM,END_DT,INDUTY_NM
0,NaN,충청북도 진천군 이월면 궁동길 51-24,19930443028350,20070119,면사랑 구수한 메밀국수,8802039211424,24개월 실온,건면,(주)면사랑,NaT,식품제조가공업
1,NaN,경기도 광주시 독고개길 156,1987036804531,20000110,종합캔디,8801696003397,제조일로부터18개월,캔디류,(주)한일제과,NaT,식품제조가공업
2,NaN,경상북도 영주시 소백로1916번길 34,201205540864,20120420,풍기홍삼愛홍삼정골드,8809359120061,제조일로부터 2년까지(실온),액상차,풍기홍삼애제조원,NaT,식품제조가공업


In [16]:
brcd1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186369 entries, 0 to 186368
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   CLSBIZ_DT         16216 non-null   object        
 1   SITE_ADDR         186359 non-null  object        
 2   PRDLST_REPORT_NO  186369 non-null  object        
 3   PRMS_DT           186369 non-null  object        
 4   PRDLST_NM         186342 non-null  object        
 5   BAR_CD            186369 non-null  object        
 6   POG_DAYCNT        181065 non-null  object        
 7   PRDLST_DCNM       186224 non-null  object        
 8   BSSH_NM           186350 non-null  object        
 9   END_DT            11804 non-null   datetime64[ns]
 10  INDUTY_NM         186365 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 15.6+ MB


### 2. 바코드, 제품명, 제조사 제외한 컬럼 삭제

In [17]:
# 컬럼 제거
brcd1.drop(
    ['POG_DAYCNT','PRMS_DT','PRDLST_REPORT_NO','INDUTY_NM','CLSBIZ_DT', 'END_DT', 'SITE_ADDR', 'PRDLST_DCNM'], 
    axis=1, 
    errors='ignore',
    inplace=True
)

In [18]:
brcd1.head(3)

,PRDLST_NM,BAR_CD,BSSH_NM
0,면사랑 구수한 메밀국수,8802039211424,(주)면사랑
1,종합캔디,8801696003397,(주)한일제과
2,풍기홍삼愛홍삼정골드,8809359120061,풍기홍삼애제조원


In [19]:
brcd1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186369 entries, 0 to 186368
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   PRDLST_NM  186342 non-null  object
 1   BAR_CD     186369 non-null  object
 2   BSSH_NM    186350 non-null  object
dtypes: object(3)
memory usage: 4.3+ MB


### 3. 제품명, 제조사 null 값 제거

In [20]:
brcd1.dropna(axis=0, how='any', inplace=True)

In [21]:
brcd1.head()

,PRDLST_NM,BAR_CD,BSSH_NM
0,면사랑 구수한 메밀국수,8802039211424,(주)면사랑
1,종합캔디,8801696003397,(주)한일제과
2,풍기홍삼愛홍삼정골드,8809359120061,풍기홍삼애제조원
3,8가지자연재료양념아삭썰은김치,8801007297224,씨제이제일제당(주)
4,하이바나나,8801073511330,삼양식품(주)유가공공장


In [22]:
brcd1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186326 entries, 0 to 186368
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   PRDLST_NM  186326 non-null  object
 1   BAR_CD     186326 non-null  object
 2   BSSH_NM    186326 non-null  object
dtypes: object(3)
memory usage: 5.7+ MB


### 4. 제조사 전처리 [혜원님]

In [23]:
brcd1['BSSH_NM'] = brcd1['BSSH_NM'].apply(lambda x: re.sub('\([\w\s&.,]+\)|㈜|[\s][\w]+공장|주식회사[\s]|[\s]주식회사|주식회사|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)공장|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)\d공장|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)제\d공장|제\d공장|\d공장|(?<=오비맥주).*|_하남','', str(x)))


In [24]:
brcd1.head()

,PRDLST_NM,BAR_CD,BSSH_NM
0,면사랑 구수한 메밀국수,8802039211424,면사랑
1,종합캔디,8801696003397,한일제과
2,풍기홍삼愛홍삼정골드,8809359120061,풍기홍삼애제조원
3,8가지자연재료양념아삭썰은김치,8801007297224,씨제이제일제당
4,하이바나나,8801073511330,삼양식품유가공공장


In [25]:
brcd1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186326 entries, 0 to 186368
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   PRDLST_NM  186326 non-null  object
 1   BAR_CD     186326 non-null  object
 2   BSSH_NM    186326 non-null  object
dtypes: object(3)
memory usage: 5.7+ MB


### 5. 제품명 전처리 [혜원님]

In [26]:
brcd1['PRDLST_NM'] = brcd1['PRDLST_NM'].apply(lambda x: re.sub('\s(?![a-z])|_|\((.*?)\)|(.*?)\?','', str(x)))


In [27]:
brcd1.head()

,PRDLST_NM,BAR_CD,BSSH_NM
0,면사랑구수한메밀국수,8802039211424,면사랑
1,종합캔디,8801696003397,한일제과
2,풍기홍삼愛홍삼정골드,8809359120061,풍기홍삼애제조원
3,8가지자연재료양념아삭썰은김치,8801007297224,씨제이제일제당
4,하이바나나,8801073511330,삼양식품유가공공장


In [28]:
brcd1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186326 entries, 0 to 186368
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   PRDLST_NM  186326 non-null  object
 1   BAR_CD     186326 non-null  object
 2   BSSH_NM    186326 non-null  object
dtypes: object(3)
memory usage: 5.7+ MB


### 6. 바코드 전처리 (띄어쓰기 없애기, 숫자 아닌 값 들어간 거 삭제)

In [29]:
brcd1['BAR_CD'] = brcd1['BAR_CD'].apply(lambda x: re.sub('\s','', str(x)))

In [30]:
brcd1.index[brcd1['BAR_CD'].str.contains("[^\d]", na=False)]

Int64Index([61205, 143303, 180608], dtype='int64')

In [31]:
brcd1.drop(brcd1.index[brcd1['BAR_CD'].str.contains("[^\d]", na=False)], axis=0, inplace=True)

In [32]:
brcd1.head()

,PRDLST_NM,BAR_CD,BSSH_NM
0,면사랑구수한메밀국수,8802039211424,면사랑
1,종합캔디,8801696003397,한일제과
2,풍기홍삼愛홍삼정골드,8809359120061,풍기홍삼애제조원
3,8가지자연재료양념아삭썰은김치,8801007297224,씨제이제일제당
4,하이바나나,8801073511330,삼양식품유가공공장


In [33]:
brcd1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186323 entries, 0 to 186368
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   PRDLST_NM  186323 non-null  object
 1   BAR_CD     186323 non-null  object
 2   BSSH_NM    186323 non-null  object
dtypes: object(3)
memory usage: 5.7+ MB


### 7. (바코드, 제품명, 제조사) 중복 삭제

In [34]:
brcd1.drop_duplicates(inplace=True)

In [35]:
brcd1.head()

,PRDLST_NM,BAR_CD,BSSH_NM
0,면사랑구수한메밀국수,8802039211424,면사랑
1,종합캔디,8801696003397,한일제과
2,풍기홍삼愛홍삼정골드,8809359120061,풍기홍삼애제조원
3,8가지자연재료양념아삭썰은김치,8801007297224,씨제이제일제당
4,하이바나나,8801073511330,삼양식품유가공공장


In [36]:
brcd1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177409 entries, 0 to 186368
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   PRDLST_NM  177409 non-null  object
 1   BAR_CD     177409 non-null  object
 2   BSSH_NM    177409 non-null  object
dtypes: object(3)
memory usage: 5.4+ MB


### 컬럼명 변경

In [37]:
brcd1_col = {'BAR_CD': 'brcd', 'PRDLST_NM':'prdt', 'BSSH_NM':'cmpny'}

In [38]:
brcd1.rename(columns=brcd1_col, inplace = True)

In [39]:
brcd1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177409 entries, 0 to 186368
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   prdt    177409 non-null  object
 1   brcd    177409 non-null  object
 2   cmpny   177409 non-null  object
dtypes: object(3)
memory usage: 5.4+ MB


# 유통바코드 데이터

In [40]:
# 유통바코드 (brcd2)
data_path_2 = os.path.join(os.pardir,'data','유통바코드_20220529.csv')
brcd2 = pd.read_csv(data_path_2, dtype=str)

In [41]:
brcd2.head(3)

,PRDLST_REPORT_NO,HTRK_PRDLST_NM,LAST_UPDT_DTM,HRNK_PRDLST_NM,BRCD_NO,PRDLST_NM,PRDT_NM,CMPNY_NM
0,201204020263,가공식품,2015-01-22 11:12:12.0,건포류,8809360172547,건어포,황금빛하늘내린황태포5미370g,용대황태연합단대륭영농조합법인
1,201204020263,가공식품,2015-01-22 11:12:12.0,건포류,8809360172530,건어포,황금빛하늘내린황태포5미320g,용대황태연합단대륭영농조합법인
2,201204020263,가공식품,2015-01-22 11:12:12.0,건포류,8809360172523,건어포,황금빛하늘내린황태포5미270g,용대황태연합단대륭영농조합법인


In [42]:
brcd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52475 entries, 0 to 52474
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   PRDLST_REPORT_NO  52475 non-null  object
 1   HTRK_PRDLST_NM    52373 non-null  object
 2   LAST_UPDT_DTM     52475 non-null  object
 3   HRNK_PRDLST_NM    52373 non-null  object
 4   BRCD_NO           52475 non-null  object
 5   PRDLST_NM         52373 non-null  object
 6   PRDT_NM           52475 non-null  object
 7   CMPNY_NM          52475 non-null  object
dtypes: object(8)
memory usage: 3.2+ MB


### 1. 바코드, 제품명, 제조사 제외한 컬럼 삭제

In [43]:
# 컬럼 제거
brcd2.drop(
    ['LAST_UPDT_DTM', 'HTRK_PRDLST_NM','PRDLST_REPORT_NO', 'PRDLST_NM','HRNK_PRDLST_NM'], 
    axis=1,
    errors='ignore',
    inplace=True
)

In [44]:
brcd2.head()

,BRCD_NO,PRDT_NM,CMPNY_NM
0,8809360172547,황금빛하늘내린황태포5미370g,용대황태연합단대륭영농조합법인
1,8809360172530,황금빛하늘내린황태포5미320g,용대황태연합단대륭영농조합법인
2,8809360172523,황금빛하늘내린황태포5미270g,용대황태연합단대륭영농조합법인
3,8809360172172,황금빛하늘내린황태포1미70g,용대황태연합단대륭영농조합법인
4,8809360172165,황금빛하늘내린황태포1미65g,용대황태연합단대륭영농조합법인


In [45]:
brcd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52475 entries, 0 to 52474
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   BRCD_NO   52475 non-null  object
 1   PRDT_NM   52475 non-null  object
 2   CMPNY_NM  52475 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


### 2. 제품명, 제조사 null 값 제거

In [46]:
brcd2.dropna(axis=0, how='any', inplace=True)

In [47]:
brcd2.head()

,BRCD_NO,PRDT_NM,CMPNY_NM
0,8809360172547,황금빛하늘내린황태포5미370g,용대황태연합단대륭영농조합법인
1,8809360172530,황금빛하늘내린황태포5미320g,용대황태연합단대륭영농조합법인
2,8809360172523,황금빛하늘내린황태포5미270g,용대황태연합단대륭영농조합법인
3,8809360172172,황금빛하늘내린황태포1미70g,용대황태연합단대륭영농조합법인
4,8809360172165,황금빛하늘내린황태포1미65g,용대황태연합단대륭영농조합법인


In [48]:
brcd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52475 entries, 0 to 52474
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   BRCD_NO   52475 non-null  object
 1   PRDT_NM   52475 non-null  object
 2   CMPNY_NM  52475 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


### 3. 제조사 전처리 [혜원님]

In [49]:
brcd2['CMPNY_NM'] = brcd2['CMPNY_NM'].apply(lambda x: re.sub('\([\w\s&.,]+\)|㈜|[\s][\w]+공장|주식회사[\s]|[\s]주식회사|주식회사|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)공장|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)\d공장|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)제\d공장|제\d공장|\d공장|(?<=오비맥주).*|_하남','', str(x)))


In [50]:
brcd2.head()

,BRCD_NO,PRDT_NM,CMPNY_NM
0,8809360172547,황금빛하늘내린황태포5미370g,용대황태연합단대륭영농조합법인
1,8809360172530,황금빛하늘내린황태포5미320g,용대황태연합단대륭영농조합법인
2,8809360172523,황금빛하늘내린황태포5미270g,용대황태연합단대륭영농조합법인
3,8809360172172,황금빛하늘내린황태포1미70g,용대황태연합단대륭영농조합법인
4,8809360172165,황금빛하늘내린황태포1미65g,용대황태연합단대륭영농조합법인


In [51]:
brcd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52475 entries, 0 to 52474
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   BRCD_NO   52475 non-null  object
 1   PRDT_NM   52475 non-null  object
 2   CMPNY_NM  52475 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


### 4. 제품명 전처리 [혜원님]

In [52]:
brcd2['PRDT_NM'] = brcd2['PRDT_NM'].apply(lambda x: re.sub('\s(?![a-z])|_|\((.*?)\)|(.*?)\?','', str(x)))


In [53]:
brcd2.head()

,BRCD_NO,PRDT_NM,CMPNY_NM
0,8809360172547,황금빛하늘내린황태포5미370g,용대황태연합단대륭영농조합법인
1,8809360172530,황금빛하늘내린황태포5미320g,용대황태연합단대륭영농조합법인
2,8809360172523,황금빛하늘내린황태포5미270g,용대황태연합단대륭영농조합법인
3,8809360172172,황금빛하늘내린황태포1미70g,용대황태연합단대륭영농조합법인
4,8809360172165,황금빛하늘내린황태포1미65g,용대황태연합단대륭영농조합법인


In [54]:
brcd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52475 entries, 0 to 52474
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   BRCD_NO   52475 non-null  object
 1   PRDT_NM   52475 non-null  object
 2   CMPNY_NM  52475 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


### 5. 바코드 전처리 (띄어쓰기 없애기, 숫자 아닌 값 들어간 거 삭제)

In [55]:
brcd2['BRCD_NO'] = brcd2['BRCD_NO'].apply(lambda x: re.sub('\s','', str(x)))

In [56]:
brcd2.index[brcd2['BRCD_NO'].str.contains("[^\d]", na=False)]

Int64Index([], dtype='int64')

In [57]:
brcd2.drop(brcd2.index[brcd2['BRCD_NO'].str.contains("[^\d]", na=False)], axis=0, inplace=True)

In [58]:
brcd2.head()

,BRCD_NO,PRDT_NM,CMPNY_NM
0,8809360172547,황금빛하늘내린황태포5미370g,용대황태연합단대륭영농조합법인
1,8809360172530,황금빛하늘내린황태포5미320g,용대황태연합단대륭영농조합법인
2,8809360172523,황금빛하늘내린황태포5미270g,용대황태연합단대륭영농조합법인
3,8809360172172,황금빛하늘내린황태포1미70g,용대황태연합단대륭영농조합법인
4,8809360172165,황금빛하늘내린황태포1미65g,용대황태연합단대륭영농조합법인


In [59]:
brcd2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52475 entries, 0 to 52474
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   BRCD_NO   52475 non-null  object
 1   PRDT_NM   52475 non-null  object
 2   CMPNY_NM  52475 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


### 5. (바코드, 제품명, 제조사) 중복 삭제

In [60]:
brcd2.drop_duplicates(inplace=True)

In [61]:
brcd2.head()

,BRCD_NO,PRDT_NM,CMPNY_NM
0,8809360172547,황금빛하늘내린황태포5미370g,용대황태연합단대륭영농조합법인
1,8809360172530,황금빛하늘내린황태포5미320g,용대황태연합단대륭영농조합법인
2,8809360172523,황금빛하늘내린황태포5미270g,용대황태연합단대륭영농조합법인
3,8809360172172,황금빛하늘내린황태포1미70g,용대황태연합단대륭영농조합법인
4,8809360172165,황금빛하늘내린황태포1미65g,용대황태연합단대륭영농조합법인


In [62]:
brcd2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51151 entries, 0 to 52474
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   BRCD_NO   51151 non-null  object
 1   PRDT_NM   51151 non-null  object
 2   CMPNY_NM  51151 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


### 컬럼명 변경

In [63]:
brcd2_col = {'BRCD_NO': 'brcd', 'PRDT_NM':'prdt', 'CMPNY_NM':'cmpny'}

In [64]:
brcd2.rename(columns=brcd2_col, inplace = True)

In [65]:
brcd2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51151 entries, 0 to 52474
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   brcd    51151 non-null  object
 1   prdt    51151 non-null  object
 2   cmpny   51151 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


# 식품 데이터 기준

In [66]:
# 수집할 컬럼명
food_col = [
    '식품명', '제조사/유통사', '지역 / 제조사', '연도', '식품상세분류', '1회제공량', '내용량_단위',
    '에너지(㎉)', '탄수화물(g)', '총 식이섬유(g)', '단백질(g)', '지방(g)', 
    '리놀레산(18:2(n-6)c)(g)', '리놀레산(18:2(n-6)c)(g)', '알파 리놀렌산(18:3(n-3))(g)', '알파 리놀렌산(18:3(n-3))(㎎)', 
    '에이코사펜타에노산(20:5(n-3))(g)', '도코사헥사에노산(22:6(n-3))(g)',
    '에이코사펜타에노산(20:5(n-3))(㎎)', '도코사헥사에노산(22:6(n-3))(㎎)', 'EPA와 DHA의 합(㎎)', '오메가 3 지방산(g)', 
    '메티오닌(㎎)', '류신(㎎)', '이소류신(㎎)', '발린(㎎)', '라이신(㎎)', '페닐알라닌(㎎)', 
    '티로신(㎎)', '트레오닌(㎎)', '트립토판(㎎)', '히스티딘(㎎)',
    '레티놀(㎍)', '비타민 A(㎍ RE)', '비타민 D(D2+D3)(㎍)', '비타민 D3(㎍)', '비타민 D1(㎍)', '토코페롤(㎎)', '토코트리에놀(㎎)',
    '비타민 E(㎎)', '비타민 E(㎎ α-TE)', '비타민 K(㎎)', '비타민 K(㎍)', '비타민 K1(㎍)', '비타민 K2(㎍)',
    '비타민 C(g)', '비타민 C(㎎)', '비타민 B1(㎎)', '비타민 B1(㎍)', '비타민 B2(㎎)', '비타민 B2(㎍)', '비타민 B2(㎎)', '비타민 B2(㎍)', 
    '나이아신(㎎ NE)', '나이아신(㎎)', '엽산(DFE)(㎍)', '비타민 B6(㎎)', '비타민 B6(㎍)', 
    '비타민 B12(㎎)', '비타민 B12(㎍)', '판토텐산(㎎)', '판토텐산(㎍)', '비오틴(㎍)',
    '칼슘(㎎)', '인(㎎)', '나트륨(㎎)', '염소(㎎)', '칼륨(㎎)', '마그네슘(㎎)',
    '철(㎎)', '철(㎍)', '아연(㎎)', '구리(㎎)', '구리(㎍)', '망간(㎎)', '망간(㎍)', '요오드(㎍)', '셀레늄(㎍)'
]

In [67]:
# DB에 저장하는 영양소 이름
ntrn_db = {
    'Calories(kcal)': ['에너지(㎉)'], 
    'Carbohydrate(g)': ['탄수화물(g)'], 
    'Dietary_fiber(g)': ['총 식이섬유(g)'], 
    'Protein(g)': ['단백질(g)'], 
    'Fat(g)': ['지방(g)'], 
    'Linoleic_acid(g)': ['리놀레산(18:2(n-6)c)(g)', '리놀레산(18:2(n-6)c)(g)'], 
    'Alpha-linolenic_acid(g)': ['알파 리놀렌산(18:3(n-3))(g)', '알파 리놀렌산(18:3(n-3))(㎎)'], 
    'EPA+DHA(mg)': [
        '에이코사펜타에노산(20:5(n-3))(g)', '도코사헥사에노산(22:6(n-3))(g)',
        '에이코사펜타에노산(20:5(n-3))(㎎)', '도코사헥사에노산(22:6(n-3))(㎎)', 'EPA와 DHA의 합(㎎)', '오메가 3 지방산(g)'
    ], 
    'Methionine(g)': ['메티오닌(㎎)'], 
    'Leucine(g)': ['류신(㎎)'], 
    'Isoleucine(g)': ['이소류신(㎎)'],
    'Valine(g)': ['발린(㎎)'], 
    'Lysine(g)': ['라이신(㎎)'], 
    'Phenylalanine+Tyrosine(g)': ['페닐알라닌(㎎)', '티로신(㎎)'], 
    'Threonine(g)': ['트레오닌(㎎)'], 
    'Tryptophan(g)': ['트립토판(㎎)'], 
    'Histidine(g)': ['히스티딘(㎎)'],
    'Vitamin_A(ug)': ['레티놀(㎍)', '비타민 A(㎍ RE)'], 
    'Vitamin_D(ug)': ['비타민 D(D2+D3)(㎍)', '비타민 D3(㎍)', '비타민 D1(㎍)'], 
    'Vitamin_E(mg)': ['토코페롤(㎎)', '토코트리에놀(㎎)', '비타민 E(㎎)', '비타민 E(㎎ α-TE)'], 
    'Vitamin_K(ug)': ['비타민 K(㎎)', '비타민 K(㎍)', '비타민 K1(㎍)', '비타민 K2(㎍)'], 
    'Vitamin_C(mg)': ['비타민 C(g)', '비타민 C(㎎)'], 
    'Vitamin_B1(mg)': ['비타민 B1(㎎)', '비타민 B1(㎍)'], 
    'Vitamin_B2(mg)': ['비타민 B2(㎎)', '비타민 B2(㎍)'], 
    'Niacin(mg)': ['나이아신(㎎ NE)', '나이아신(㎎)'], 
    'Vitamin_B6(mg)': ['비타민 B6(㎎)', '비타민 B6(㎍)'], 
    'Folic_acid(ug)': ['엽산(DFE)(㎍)'],
    'Vitamin_B12(ug)': ['비타민 B12(㎎)', '비타민 B12(㎍)'], 
    'Pantothenic_acid(mg)': ['판토텐산(㎎)', '판토텐산(㎍)'], 
    'Biotin(ug)': ['비오틴(㎍)'], 
    'Calcium(mg)': ['칼슘(㎎)'], 
    'Phosphorus(mg)': ['인(㎎)'], 
    'Sodium(mg)': ['나트륨(㎎)'], 
    'Chloride(mg)': ['염소(㎎)'], 
    'Potassium(mg)': ['칼륨(㎎)'], 
    'Magnesium(mg)': ['마그네슘(㎎)'],
    'Iron(mg)': ['철(㎎)', '철(㎍)'], 
    'Zinc(mg)': ['아연(㎎)'], 
    'Copper(ug)': ['구리(㎎)', '구리(㎍)'], 
    'Manganese(mg)': ['망간(㎎)', '망간(㎍)'], 
    'Iodine(ug)': ['요오드(㎍)'], 
    'Selenium(ug)': ['셀레늄(㎍)']
}

In [68]:
# ntrn_db를 사용하여 컬럼명 바꾸는 dict 만들기
def get_ntrn_data(ntrn_dict):
    ntrn_dt = dict()
    for k, v in ntrn_dict.items():
        for i in range(len(v)):
            ntrn_dt[v[i]] = k
    return ntrn_dt

ntrn_dt = get_ntrn_data(ntrn_db)

In [69]:
other_col = {
    '식품명':'food_name', '식품상세분류':'food_cat', '제조사/유통사':'cmpny', '연도':'year',
    '지역 / 제조사':'cmpny','1회제공량':'serving_amount', '내용량_단위':'serving_unit'
}
col_to = dict(other_col, **ntrn_dt)

In [70]:
col_to

{'식품명': 'food_name',
 '식품상세분류': 'food_cat',
 '제조사/유통사': 'cmpny',
 '연도': 'year',
 '지역 / 제조사': 'cmpny',
 '1회제공량': 'serving_amount',
 '내용량_단위': 'serving_unit',
 '에너지(㎉)': 'Calories(kcal)',
 '탄수화물(g)': 'Carbohydrate(g)',
 '총 식이섬유(g)': 'Dietary_fiber(g)',
 '단백질(g)': 'Protein(g)',
 '지방(g)': 'Fat(g)',
 '리놀레산(18:2(n-6)c)(g)': 'Linoleic_acid(g)',
 '알파 리놀렌산(18:3(n-3))(g)': 'Alpha-linolenic_acid(g)',
 '알파 리놀렌산(18:3(n-3))(㎎)': 'Alpha-linolenic_acid(g)',
 '에이코사펜타에노산(20:5(n-3))(g)': 'EPA+DHA(mg)',
 '도코사헥사에노산(22:6(n-3))(g)': 'EPA+DHA(mg)',
 '에이코사펜타에노산(20:5(n-3))(㎎)': 'EPA+DHA(mg)',
 '도코사헥사에노산(22:6(n-3))(㎎)': 'EPA+DHA(mg)',
 'EPA와 DHA의 합(㎎)': 'EPA+DHA(mg)',
 '오메가 3 지방산(g)': 'EPA+DHA(mg)',
 '메티오닌(㎎)': 'Methionine(g)',
 '류신(㎎)': 'Leucine(g)',
 '이소류신(㎎)': 'Isoleucine(g)',
 '발린(㎎)': 'Valine(g)',
 '라이신(㎎)': 'Lysine(g)',
 '페닐알라닌(㎎)': 'Phenylalanine+Tyrosine(g)',
 '티로신(㎎)': 'Phenylalanine+Tyrosine(g)',
 '트레오닌(㎎)': 'Threonine(g)',
 '트립토판(㎎)': 'Tryptophan(g)',
 '히스티딘(㎎)': 'Histidine(g)',
 '레티놀(㎍)': 'Vitamin_A(

# 가공식품 데이터

### 1. 필요 없는 컬럼 삭제

In [71]:
not_in_p = [
    '지역 / 제조사','레티놀(㎍)', '이소류신(㎎)', '나이아신(㎎)', '철(㎍)', '알파 리놀렌산(18:3(n-3))(g)', 
    '트레오닌(㎎)', '토코트리에놀(㎎)', '발린(㎎)', '토코페롤(㎎)', '도코사헥사에노산(22:6(n-3))(g)', 
    '페닐알라닌(㎎)', '메티오닌(㎎)', '에이코사펜타에노산(20:5(n-3))(g)', '라이신(㎎)', '티로신(㎎)'
]
# 수집할 컬럼명
food_col_p = [col for col in food_col if col not in not_in_p]

In [72]:
# 가공식품 데이터
food_p_path = os.path.join(os.pardir,'data','통합_식품영양성분DB_가공식품_20220502.csv')
food_p = pd.read_csv(food_p_path, dtype=str, na_values='-',usecols=food_col_p)

In [73]:
food_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77797 entries, 0 to 77796
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   식품명                      77797 non-null  object
 1   연도                       77797 non-null  object
 2   제조사/유통사                  77797 non-null  object
 3   식품상세분류                   77797 non-null  object
 4   1회제공량                    77797 non-null  object
 5   내용량_단위                   76835 non-null  object
 6   에너지(㎉)                   77733 non-null  object
 7   단백질(g)                   75119 non-null  object
 8   지방(g)                    73565 non-null  object
 9   탄수화물(g)                  74992 non-null  object
 10  총 식이섬유(g)                2756 non-null   object
 11  칼슘(㎎)                    3307 non-null   object
 12  철(㎎)                     2217 non-null   object
 13  마그네슘(㎎)                  283 non-null    object
 14  인(㎎)                     495 non-null 

### 2. 컬럼명 변경, 단위 환산

In [74]:
# 단위 환산
# mg -> g
# ug -> g (필요 없을 듯)
# g -> mg
# ug -> mg
# g -> ug (필요 없을 듯)
# mg -> ug
def convert_units(fu, tu):
    # to (g)
    if '(g' in tu:
        # from (mg, ㎎)
        if ('(mg' in fu) | ('㎎' in fu):
            return 0.001
        else:
            pass
    # to (mg)
    elif '(mg' in tu:
        # from (g)
        if '(g' in fu:
            return 1000
        # from (ug, ㎍)
        elif ('(ug' in fu) | ('㎍' in fu):
            return 0.001
        else:
            pass
    # to (ug)
    elif 'ug' in tu:
        # from (mg, ㎎)
        if ('(mg' in fu) | ('㎎' in fu):
            return 1000
        else:
            pass
    else:
        pass
    return 1

In [75]:
# 단위 환산 하면서, 컬럼명 변경
def preprocessing_food(df, col_to):
    df_cols = df.columns
    for c in df_cols:
        f_unit = re.findall('\([\D\s]*\)', c)
        t_unit = re.findall('\([\D\s]*\)', col_to[c])
        if f_unit:
            convert_u = convert_units(f_unit[0], t_unit[0])
            # print(c, f_unit[0], '//',col_to[c], t_unit[0])
            try:
                df[col_to[c]] += pd.to_numeric(df[c], errors='coerce').fillna(0) * convert_u
            except:
                df[col_to[c]] = pd.to_numeric(df[c], errors='coerce').fillna(0) * convert_u
            df.drop([c], axis=1, inplace=True)
        else:
            df[col_to[c]] = df[c]
            df.drop([c], axis=1, inplace=True)
    return df


In [76]:
food_p_p = preprocessing_food(food_p, col_to)

In [77]:
food_p_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77797 entries, 0 to 77796
Data columns (total 42 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   food_name                77797 non-null  object 
 1   year                     77797 non-null  object 
 2   cmpny                    77797 non-null  object 
 3   food_cat                 77797 non-null  object 
 4   serving_amount           77797 non-null  object 
 5   serving_unit             76835 non-null  object 
 6   Calories(kcal)           77797 non-null  float64
 7   Protein(g)               77797 non-null  float64
 8   Fat(g)                   77797 non-null  float64
 9   Carbohydrate(g)          77797 non-null  float64
 10  Dietary_fiber(g)         77797 non-null  float64
 11  Calcium(mg)              77797 non-null  float64
 12  Iron(mg)                 77797 non-null  float64
 13  Magnesium(mg)            77797 non-null  float64
 14  Phosphorus(mg)        

### 3. 제조사 전처리 [혜원님]

In [78]:
food_p_p['cmpny'] = food_p_p['cmpny'].apply(lambda x: re.sub('\([\w\s&.,]+\)|㈜|[\s][\w]+공장|주식회사[\s]|[\s]주식회사|주식회사|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)공장|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)\d공장|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)제\d공장|제\d공장|\d공장|(?<=오비맥주).*|_하남','', str(x)))


In [79]:
food_p_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77797 entries, 0 to 77796
Data columns (total 42 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   food_name                77797 non-null  object 
 1   year                     77797 non-null  object 
 2   cmpny                    77797 non-null  object 
 3   food_cat                 77797 non-null  object 
 4   serving_amount           77797 non-null  object 
 5   serving_unit             76835 non-null  object 
 6   Calories(kcal)           77797 non-null  float64
 7   Protein(g)               77797 non-null  float64
 8   Fat(g)                   77797 non-null  float64
 9   Carbohydrate(g)          77797 non-null  float64
 10  Dietary_fiber(g)         77797 non-null  float64
 11  Calcium(mg)              77797 non-null  float64
 12  Iron(mg)                 77797 non-null  float64
 13  Magnesium(mg)            77797 non-null  float64
 14  Phosphorus(mg)        

### 4. 제품명 전처리 [혜원님]

In [80]:
food_p_p['food_name'] = food_p_p['food_name'].apply(lambda x: re.sub('\s(?![a-z])|_|\((.*?)\)|(.*?)\?','', str(x)))


In [81]:
food_p_p.head()

,food_name,year,cmpny,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),Carbohydrate(g),...,Biotin(ug),Folic_acid(ug),Vitamin_B12(ug),Vitamin_C(mg),Leucine(g),Tryptophan(g),Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg)
0,구운인절미,2017,해태가루비,스낵과자,50,g,175.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,코펜라스버터쿠키,2017,매크로통상,쿠키,50,g,254.8,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,로아커웨하스바닐라,2017,델리팜,웨이퍼,30,g,155.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,맛미강정,2017,왕궁병과,한과류,45,g,190.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,찹쌀약과,2017,성진식품,한과류,40,g,155.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
food_p_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77797 entries, 0 to 77796
Data columns (total 42 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   food_name                77797 non-null  object 
 1   year                     77797 non-null  object 
 2   cmpny                    77797 non-null  object 
 3   food_cat                 77797 non-null  object 
 4   serving_amount           77797 non-null  object 
 5   serving_unit             76835 non-null  object 
 6   Calories(kcal)           77797 non-null  float64
 7   Protein(g)               77797 non-null  float64
 8   Fat(g)                   77797 non-null  float64
 9   Carbohydrate(g)          77797 non-null  float64
 10  Dietary_fiber(g)         77797 non-null  float64
 11  Calcium(mg)              77797 non-null  float64
 12  Iron(mg)                 77797 non-null  float64
 13  Magnesium(mg)            77797 non-null  float64
 14  Phosphorus(mg)        

### 5. (제품명, 제조사, 년도, 1회제공량) 중복 삭제

In [83]:
cols = ['food_name','cmpny','year','serving_amount']

In [84]:
drop_index = food_p_p.loc[:,cols][food_p_p.loc[:,cols].duplicated()].index

In [85]:
food_p_p.drop(drop_index, axis=0, inplace=True)

In [86]:
food_p_p[food_p_p['food_name'] == '새우깡']

,food_name,year,cmpny,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),Carbohydrate(g),...,Biotin(ug),Folic_acid(ug),Vitamin_B12(ug),Vitamin_C(mg),Leucine(g),Tryptophan(g),Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg)
181,새우깡,2016,농심,스낵과자,45,g,225.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13221,새우깡,2019,농심,스낵과자,30,g,145.0,2.0,7.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19298,새우깡,2020,농심,스낵과자,30,g,150.0,2.0,8.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47185,새우깡,2021,농심,스낵과자,90,g,465.0,6.0,25.0,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
food_p_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76192 entries, 0 to 77796
Data columns (total 42 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   food_name                76192 non-null  object 
 1   year                     76192 non-null  object 
 2   cmpny                    76192 non-null  object 
 3   food_cat                 76192 non-null  object 
 4   serving_amount           76192 non-null  object 
 5   serving_unit             75306 non-null  object 
 6   Calories(kcal)           76192 non-null  float64
 7   Protein(g)               76192 non-null  float64
 8   Fat(g)                   76192 non-null  float64
 9   Carbohydrate(g)          76192 non-null  float64
 10  Dietary_fiber(g)         76192 non-null  float64
 11  Calcium(mg)              76192 non-null  float64
 12  Iron(mg)                 76192 non-null  float64
 13  Magnesium(mg)            76192 non-null  float64
 14  Phosphorus(mg)        

### 6. (제품명, 제조사, 1회제공량) 중복 → 최근 년도만 남기기

In [88]:
food_p_p.drop_duplicates(subset=['food_name','cmpny','serving_amount'],keep='last', inplace=True)

In [89]:
food_p_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62630 entries, 0 to 77796
Data columns (total 42 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   food_name                62630 non-null  object 
 1   year                     62630 non-null  object 
 2   cmpny                    62630 non-null  object 
 3   food_cat                 62630 non-null  object 
 4   serving_amount           62630 non-null  object 
 5   serving_unit             62276 non-null  object 
 6   Calories(kcal)           62630 non-null  float64
 7   Protein(g)               62630 non-null  float64
 8   Fat(g)                   62630 non-null  float64
 9   Carbohydrate(g)          62630 non-null  float64
 10  Dietary_fiber(g)         62630 non-null  float64
 11  Calcium(mg)              62630 non-null  float64
 12  Iron(mg)                 62630 non-null  float64
 13  Magnesium(mg)            62630 non-null  float64
 14  Phosphorus(mg)        

# 음식 데이터

### 1. 필요 없는 컬럼 삭제

In [91]:
not_in_f = [
    '제조사/유통사', '비타민 D(D2+D3)(㎍)', '비타민 D1(㎍)', '나이아신(㎎ NE)', '트립토판(㎎)', '비타민 B6(㎍)', 
    '판토텐산(㎎)', '비타민 B1(㎍)', '비타민 E(㎎)', '비오틴(㎍)', '비타민 K(㎎)', '비타민 E(㎎ α-TE)', 
    '비타민 C(g)', '비타민 K(㎍)', '비타민 B2(㎍)', '판토텐산(㎍)', '비타민 B6(㎎)', '비타민 A(㎍ RE)', 
    '망간(㎍)', '오메가 3 지방산(g)', '도코사헥사에노산(22:6(n-3))(㎎)', '에이코사펜타에노산(20:5(n-3))(㎎)', 
    '비타민 B12(㎎)', '요오드(㎍)', '염소(㎎)', '비타민 K1(㎍)', '비타민 K2(㎍)', '구리(㎍)', 
    'EPA와 DHA의 합(㎎)', '알파 리놀렌산(18:3(n-3))(㎎)'
]
# 수집할 컬럼명
food_col_f = [col for col in food_col if col not in not_in_f]

In [92]:
food_col_f

['식품명',
 '지역 / 제조사',
 '연도',
 '식품상세분류',
 '1회제공량',
 '내용량_단위',
 '에너지(㎉)',
 '탄수화물(g)',
 '총 식이섬유(g)',
 '단백질(g)',
 '지방(g)',
 '리놀레산(18:2(n-6)c)(g)',
 '리놀레산(18:2(n-6)c)(g)',
 '알파 리놀렌산(18:3(n-3))(g)',
 '에이코사펜타에노산(20:5(n-3))(g)',
 '도코사헥사에노산(22:6(n-3))(g)',
 '메티오닌(㎎)',
 '류신(㎎)',
 '이소류신(㎎)',
 '발린(㎎)',
 '라이신(㎎)',
 '페닐알라닌(㎎)',
 '티로신(㎎)',
 '트레오닌(㎎)',
 '히스티딘(㎎)',
 '레티놀(㎍)',
 '비타민 D3(㎍)',
 '토코페롤(㎎)',
 '토코트리에놀(㎎)',
 '비타민 C(㎎)',
 '비타민 B1(㎎)',
 '비타민 B2(㎎)',
 '비타민 B2(㎎)',
 '나이아신(㎎)',
 '엽산(DFE)(㎍)',
 '비타민 B12(㎍)',
 '칼슘(㎎)',
 '인(㎎)',
 '나트륨(㎎)',
 '칼륨(㎎)',
 '마그네슘(㎎)',
 '철(㎎)',
 '철(㎍)',
 '아연(㎎)',
 '구리(㎎)',
 '망간(㎎)',
 '셀레늄(㎍)']

In [93]:
# 음식 데이터
food_f_path = os.path.join(os.pardir,'data','통합_식품영양성분DB_음식_20220120.csv')
food_f = pd.read_csv(food_f_path, dtype=str, na_values='-',usecols=food_col_f)

In [94]:
food_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7704 entries, 0 to 7703
Data columns (total 45 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   식품명                      7704 non-null   object
 1   연도                       7704 non-null   object
 2   지역 / 제조사                 7704 non-null   object
 3   식품상세분류                   7704 non-null   object
 4   1회제공량                    7704 non-null   object
 5   내용량_단위                   7704 non-null   object
 6   에너지(㎉)                   7686 non-null   object
 7   단백질(g)                   7504 non-null   object
 8   지방(g)                    1585 non-null   object
 9   탄수화물(g)                  1831 non-null   object
 10  총 식이섬유(g)                839 non-null    object
 11  칼슘(㎎)                    839 non-null    object
 12  철(㎎)                     548 non-null    object
 13  철(㎍)                     291 non-null    object
 14  마그네슘(㎎)                  839 non-null   

### 2. 컬럼명 변경, 단위 환산

In [95]:
# 단위 환산
# mg -> g
# ug -> g (필요 없을 듯)
# g -> mg
# ug -> mg
# g -> ug (필요 없을 듯)
# mg -> ug
def convert_units(fu, tu):
    # to (g)
    if '(g' in tu:
        # from (mg, ㎎)
        if ('(mg' in fu) | ('㎎' in fu):
            return 0.001
        else:
            pass
    # to (mg)
    elif '(mg' in tu:
        # from (g)
        if '(g' in fu:
            return 1000
        # from (ug, ㎍)
        elif ('(ug' in fu) | ('㎍' in fu):
            return 0.001
        else:
            pass
    # to (ug)
    elif 'ug' in tu:
        # from (mg, ㎎)
        if ('(mg' in fu) | ('㎎' in fu):
            return 1000
        else:
            pass
    else:
        pass
    return 1

In [96]:
# 단위 환산 하면서, 컬럼명 변경
def preprocessing_food(df, col_to):
    df_cols = df.columns
    for c in df_cols:
        f_unit = re.findall('\([\D\s]*\)', c)
        t_unit = re.findall('\([\D\s]*\)', col_to[c])
        if f_unit:
            convert_u = convert_units(f_unit[0], t_unit[0])
            # print(c, f_unit[0], '//',col_to[c], t_unit[0])
            try:
                df[col_to[c]] += pd.to_numeric(df[c], errors='coerce').fillna(0) * convert_u
            except:
                df[col_to[c]] = pd.to_numeric(df[c], errors='coerce').fillna(0) * convert_u
            df.drop([c], axis=1, inplace=True)
        else:
            df[col_to[c]] = df[c]
            df.drop([c], axis=1, inplace=True)
    return df


In [97]:
food_f_p = preprocessing_food(food_f, col_to)

In [98]:
food_f_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7704 entries, 0 to 7703
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   food_name                  7704 non-null   object 
 1   year                       7704 non-null   object 
 2   cmpny                      7704 non-null   object 
 3   food_cat                   7704 non-null   object 
 4   serving_amount             7704 non-null   object 
 5   serving_unit               7704 non-null   object 
 6   Calories(kcal)             7704 non-null   float64
 7   Protein(g)                 7704 non-null   float64
 8   Fat(g)                     7704 non-null   float64
 9   Carbohydrate(g)            7704 non-null   float64
 10  Dietary_fiber(g)           7704 non-null   float64
 11  Calcium(mg)                7704 non-null   float64
 12  Iron(mg)                   7704 non-null   float64
 13  Magnesium(mg)              7704 non-null   float

### 3. 제조사 전처리 [혜원님]

In [99]:
food_f_p['cmpny'] = food_f_p['cmpny'].apply(lambda x: re.sub('\([\w\s&.,]+\)|㈜|[\s][\w]+공장|주식회사[\s]|[\s]주식회사|주식회사|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)공장|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)\d공장|(평택|기흥|영동|고성|송탄|안성|안양|오산|거창|익산|양주|김해|천안|세종|진천|달성|경산|원주|청주|천안|광주|횡성유가공|향남|부안|경주|수원|경남|성환|군산|안산|횡성|성남|춘천|양산|부산|홍성|시화|탄진|상하|증평|영주|천안신|남원|조치원|포승|파스퇴르|음성|금산|광혜원|김제|강원)제\d공장|제\d공장|\d공장|(?<=오비맥주).*|_하남','', str(x)))


In [100]:
food_f_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7704 entries, 0 to 7703
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   food_name                  7704 non-null   object 
 1   year                       7704 non-null   object 
 2   cmpny                      7704 non-null   object 
 3   food_cat                   7704 non-null   object 
 4   serving_amount             7704 non-null   object 
 5   serving_unit               7704 non-null   object 
 6   Calories(kcal)             7704 non-null   float64
 7   Protein(g)                 7704 non-null   float64
 8   Fat(g)                     7704 non-null   float64
 9   Carbohydrate(g)            7704 non-null   float64
 10  Dietary_fiber(g)           7704 non-null   float64
 11  Calcium(mg)                7704 non-null   float64
 12  Iron(mg)                   7704 non-null   float64
 13  Magnesium(mg)              7704 non-null   float

### 4. 제품명 전처리 [혜원님]

In [101]:
food_f_p['food_name'] = food_f_p['food_name'].apply(lambda x: re.sub('\s(?![a-z])|_|\((.*?)\)|(.*?)\?','', str(x)))


In [102]:
food_f_p.head()

,food_name,year,cmpny,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),Carbohydrate(g),...,Leucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g),Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg)
0,꿩불고기,2019,충주,육류구이,500,g,368.800000,33.500000,8.500000,39.700000,...,2.328599,2.484750,0.64519,2.194733,1.410841,1.367688,0.848539,3.236018,0.656723,23.426982
1,닭갈비,2019,전국,육류구이,400,g,595.610000,45.900000,25.800000,44.900000,...,3.245405,3.593312,0.87804,3.140655,2.019939,1.966315,1.198582,4.851294,0.443453,15.925505
2,닭갈비,2019,춘천,육류구이,300,g,558.470000,45.500000,31.600000,23.100000,...,3.050299,3.210690,0.97059,3.070835,1.873143,1.707208,1.133533,5.434226,0.390666,26.856086
3,닭꼬치,2019,전국,육류구이,70,g,176.722661,11.561723,8.565014,13.347661,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.900000,0.200000,0.000000
4,더덕구이,2019,전국,채소류구이,100,g,184.000000,3.100000,5.200000,31.100000,...,0.126530,0.090720,0.01636,0.116640,0.070970,0.087910,0.044560,2.300000,0.400000,0.000000


In [103]:
food_f_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7704 entries, 0 to 7703
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   food_name                  7704 non-null   object 
 1   year                       7704 non-null   object 
 2   cmpny                      7704 non-null   object 
 3   food_cat                   7704 non-null   object 
 4   serving_amount             7704 non-null   object 
 5   serving_unit               7704 non-null   object 
 6   Calories(kcal)             7704 non-null   float64
 7   Protein(g)                 7704 non-null   float64
 8   Fat(g)                     7704 non-null   float64
 9   Carbohydrate(g)            7704 non-null   float64
 10  Dietary_fiber(g)           7704 non-null   float64
 11  Calcium(mg)                7704 non-null   float64
 12  Iron(mg)                   7704 non-null   float64
 13  Magnesium(mg)              7704 non-null   float

### 5. (제품명, 제조사, 년도, 1회제공량) 중복 삭제

In [104]:
cols = ['food_name','cmpny','year','serving_amount']

In [105]:
drop_index = food_f_p.loc[:,cols][food_f_p.loc[:,cols].duplicated()].index

In [106]:
food_f_p.drop(drop_index, axis=0, inplace=True)

In [107]:
food_f_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7572 entries, 0 to 7703
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   food_name                  7572 non-null   object 
 1   year                       7572 non-null   object 
 2   cmpny                      7572 non-null   object 
 3   food_cat                   7572 non-null   object 
 4   serving_amount             7572 non-null   object 
 5   serving_unit               7572 non-null   object 
 6   Calories(kcal)             7572 non-null   float64
 7   Protein(g)                 7572 non-null   float64
 8   Fat(g)                     7572 non-null   float64
 9   Carbohydrate(g)            7572 non-null   float64
 10  Dietary_fiber(g)           7572 non-null   float64
 11  Calcium(mg)                7572 non-null   float64
 12  Iron(mg)                   7572 non-null   float64
 13  Magnesium(mg)              7572 non-null   float

### 6. (제품명, 제조사, 1회제공량) 중복 → 최근 년도만 남기기

In [108]:
food_f_p.drop_duplicates(subset=['food_name','cmpny','serving_amount'],keep='last', inplace=True)

In [109]:
food_f_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7569 entries, 0 to 7703
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   food_name                  7569 non-null   object 
 1   year                       7569 non-null   object 
 2   cmpny                      7569 non-null   object 
 3   food_cat                   7569 non-null   object 
 4   serving_amount             7569 non-null   object 
 5   serving_unit               7569 non-null   object 
 6   Calories(kcal)             7569 non-null   float64
 7   Protein(g)                 7569 non-null   float64
 8   Fat(g)                     7569 non-null   float64
 9   Carbohydrate(g)            7569 non-null   float64
 10  Dietary_fiber(g)           7569 non-null   float64
 11  Calcium(mg)                7569 non-null   float64
 12  Iron(mg)                   7569 non-null   float64
 13  Magnesium(mg)              7569 non-null   float

# 데이터 합치기

## 바코드 데이터

In [110]:
brcd = pd.concat([brcd1,brcd2])

In [111]:
brcd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228560 entries, 0 to 52474
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   prdt    228560 non-null  object
 1   brcd    228560 non-null  object
 2   cmpny   228560 non-null  object
dtypes: object(3)
memory usage: 7.0+ MB


In [112]:
brcd[brcd['brcd'].duplicated()]

,prdt,brcd,cmpny
186,해당돌자반,8809395520009,해당식품
479,쌀강정,8809064050028,아름식품
585,파채소스,8809315110013,청맥식품
602,푸르메다제주감귤꽃홍차,8809401990215,푸르메다
684,머스타드소스,8809265340072,다솔식품
...,...,...,...
52453,오리온700초코송이41g,8801117432201,오리온
52460,영지꿀차,8809017900172,이레홍삼공사
52462,참고을태양초진고추장골드14kg,8809056511254,참고을
52463,레디조10개입 gift,8809258540144,보은제약


In [113]:
brcd[brcd['brcd'] == '8809395520009']

,prdt,brcd,cmpny
185,해당광천파래자반볶음,8809395520009,해당식품
186,해당돌자반,8809395520009,해당식품
43895,해당식품해당파래자반볶음70g,8809395520009,해당식품


## 음식 데이터

In [114]:
food = pd.concat([food_p_p,food_f_p])

In [115]:
food.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70199 entries, 0 to 7703
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   food_name                  70199 non-null  object 
 1   year                       70199 non-null  object 
 2   cmpny                      70199 non-null  object 
 3   food_cat                   70199 non-null  object 
 4   serving_amount             70199 non-null  object 
 5   serving_unit               69845 non-null  object 
 6   Calories(kcal)             70199 non-null  float64
 7   Protein(g)                 70199 non-null  float64
 8   Fat(g)                     70199 non-null  float64
 9   Carbohydrate(g)            70199 non-null  float64
 10  Dietary_fiber(g)           70199 non-null  float64
 11  Calcium(mg)                70199 non-null  float64
 12  Iron(mg)                   70199 non-null  float64
 13  Magnesium(mg)              70199 non-null  floa

### 음식 데이터 csv

In [186]:
food.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70199 entries, 0 to 7703
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   food_name                  70199 non-null  object 
 1   year                       70199 non-null  object 
 2   cmpny                      70199 non-null  object 
 3   food_cat                   70199 non-null  object 
 4   serving_amount             70199 non-null  object 
 5   serving_unit               69845 non-null  object 
 6   Calories(kcal)             70199 non-null  float64
 7   Protein(g)                 70199 non-null  float64
 8   Fat(g)                     70199 non-null  float64
 9   Carbohydrate(g)            70199 non-null  float64
 10  Dietary_fiber(g)           70199 non-null  float64
 11  Calcium(mg)                70199 non-null  float64
 12  Iron(mg)                   70199 non-null  float64
 13  Magnesium(mg)              70199 non-null  floa

In [191]:
# 연도 순으로 정렬
food_test_df = food.sort_values(by=['year'], ignore_index=True)

In [192]:
food_test_df.head()

,food_name,year,cmpny,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),Carbohydrate(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
0,쿱호박고구마스틱,2016,성연식품,스낵과자,34,g,150.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,우리밀김치라면,2016,새롬식품,유탕면,113,g,500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,유기농든든한아침고소한시리얼,2016,산들촌,기타과자,35,g,151.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,우리아이착한새우,2016,풍전나이스제과,스낵과자,30,g,140.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,코카콜라,2016,코카콜라음료,탄산음료,200,g,92.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
food_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70199 entries, 0 to 70198
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   food_name                  70199 non-null  object 
 1   year                       70199 non-null  object 
 2   cmpny                      70199 non-null  object 
 3   food_cat                   70199 non-null  object 
 4   serving_amount             70199 non-null  object 
 5   serving_unit               69845 non-null  object 
 6   Calories(kcal)             70199 non-null  float64
 7   Protein(g)                 70199 non-null  float64
 8   Fat(g)                     70199 non-null  float64
 9   Carbohydrate(g)            70199 non-null  float64
 10  Dietary_fiber(g)           70199 non-null  float64
 11  Calcium(mg)                70199 non-null  float64
 12  Iron(mg)                   70199 non-null  float64
 13  Magnesium(mg)              70199 non-null  flo

In [196]:
# 중복 확인
food_test_df.iloc[:,:5][food_test_df.duplicated(subset=['food_name','cmpny','serving_amount'])]

,food_name,year,cmpny,food_cat,serving_amount
10434,콘샐러드,2019,KFC,샐러드,100
28256,바닐라,2020,비알코리아배스킨라빈스,아이스크림,115
28271,자모카아몬드훠지,2020,비알코리아배스킨라빈스,아이스크림,115
28274,쫀떡궁합,2020,비알코리아배스킨라빈스,아이스크림,115
28275,초콜릿무스,2020,비알코리아배스킨라빈스,아이스크림,115
28281,골든애플요거트,2020,비알코리아배스킨라빈스,샤베트,115
28283,마법사의할로윈,2020,비알코리아배스킨라빈스,아이스크림,115
28286,오레오쿠키앤크림치즈,2020,비알코리아배스킨라빈스,아이스크림,115
28289,너는참달고나,2020,비알코리아배스킨라빈스,아이스크림,115
28301,그린티,2020,비알코리아배스킨라빈스,아이스크림,115


In [198]:
food_test_df[food_test_df['food_name'] =='사랑에빠진딸기']

,food_name,year,cmpny,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),Carbohydrate(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
8053,사랑에빠진딸기,2019,비알코리아배스킨라빈스,아이스크림류,115,mL,285.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28334,사랑에빠진딸기,2020,비알코리아배스킨라빈스,아이스밀크,115,g,292.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [199]:
# 중복 제거 : ['food_name','cmpny','serving_amount']
food_test_df.drop_duplicates(subset=['food_name','cmpny','serving_amount'], 
                             keep='last', 
                             inplace=True, 
                             ignore_index=True)

In [204]:
food_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70169 entries, 0 to 70168
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   food_name                  70169 non-null  object 
 1   year                       70169 non-null  object 
 2   cmpny                      70169 non-null  object 
 3   food_cat                   70169 non-null  object 
 4   serving_amount             70169 non-null  object 
 5   serving_unit               69815 non-null  object 
 6   Calories(kcal)             70169 non-null  float64
 7   Protein(g)                 70169 non-null  float64
 8   Fat(g)                     70169 non-null  float64
 9   Carbohydrate(g)            70169 non-null  float64
 10  Dietary_fiber(g)           70169 non-null  float64
 11  Calcium(mg)                70169 non-null  float64
 12  Iron(mg)                   70169 non-null  float64
 13  Magnesium(mg)              70169 non-null  flo

In [201]:
# 중복 확인
food_test_df.iloc[:,:5][food_test_df.duplicated(subset=['food_name','cmpny'])]

,food_name,year,cmpny,food_cat,serving_amount
2211,청정원카레여왕토마토&치킨,2017,대상,카레(커리),200
2218,청정원렌지짱미트볼,2017,대상,즉석조리식품,200
2307,그대로짜장,2017,오뚜기,소스,200
2383,3분낙지덮밥소스,2017,오뚜기,소스,150
3375,그린비아알디플러스,2017,정식품,환자용 식품,200
...,...,...,...,...,...
70126,무농약쌀로만든초코칩쿠키,2021,리리식품,쿠키,100
70129,모나카,2021,리리식품,기타빵류,35
70142,머랭쿠키,2021,마늘빵집,스낵과자,80
70152,록키로드쿠키,2021,리치코리아 유한회사,쿠키,85


In [203]:
food_test_df[food_test_df['food_name'] =='모나카']

,food_name,year,cmpny,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),Carbohydrate(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
4459,모나카,2017,청우식품,기타과자,26,g,83.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
54435,모나카,2021,청우식품,기타빵류,100,g,315.0,3.0,0.3,75.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
70112,모나카,2021,리리식품,기타빵류,30,g,93.0,1.0,0.0,22.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
70129,모나카,2021,리리식품,기타빵류,35,g,123.0,3.0,0.0,27.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [205]:
# 용량 순으로 정렬
food_test_df.sort_values(by=['serving_amount'], ignore_index=True, inplace=True)

In [206]:
# 중복 제거 : ['food_name','cmpny']
food_test_df.drop_duplicates(subset=['food_name','cmpny'], 
                             keep='first', 
                             inplace=True, 
                             ignore_index=True)

In [207]:
food_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66318 entries, 0 to 66317
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   food_name                  66318 non-null  object 
 1   year                       66318 non-null  object 
 2   cmpny                      66318 non-null  object 
 3   food_cat                   66318 non-null  object 
 4   serving_amount             66318 non-null  object 
 5   serving_unit               66000 non-null  object 
 6   Calories(kcal)             66318 non-null  float64
 7   Protein(g)                 66318 non-null  float64
 8   Fat(g)                     66318 non-null  float64
 9   Carbohydrate(g)            66318 non-null  float64
 10  Dietary_fiber(g)           66318 non-null  float64
 11  Calcium(mg)                66318 non-null  float64
 12  Iron(mg)                   66318 non-null  float64
 13  Magnesium(mg)              66318 non-null  flo

In [208]:
food.iloc[:,:5][food.duplicated(subset=['food_name','food_cat'])]

,food_name,year,cmpny,food_cat,serving_amount
185,생칼국수,2016,미농식품,생면,188
281,옛날자른당면,2016,동순덕식품,건면,25
382,우유식빵,2016,현대그린푸드,식빵,105
419,생칼국수,2016,영일식품,생면,167
469,옛날당면,2016,귀일농산,건면,100
...,...,...,...,...,...
7604,돼지고기김치찌개,2020,쿡솜씨,육류찌개.전골,100
7607,갈치조림,2020,쿡솜씨,어패류조림,100
7608,고등어조림,2020,쿡솜씨,어패류조림,100
7626,소고기샤브샤브,2020,프레시지,육류찌개.전골,100


In [209]:
food[food['food_name'] =='생칼국수']

,food_name,year,cmpny,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),Carbohydrate(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
121,생칼국수,2016,대판식품,생면,167,g,464.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
185,생칼국수,2016,미농식품,생면,188,g,525.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
419,생칼국수,2016,영일식품,생면,167,g,487.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1258,생칼국수,2018,대자연식품,생면,100,g,270.0,6.7,1.1,58.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4125,생칼국수,2017,풀무원,생면,100,g,262.0,5.0,0.7,59.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11714,생칼국수,2019,면사랑,생면,167,g,465.0,13.0,2.8,97.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11720,생칼국수,2019,초록바구니,생면,100,g,285.0,6.6,1.0,62.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11722,생칼국수,2019,미정,생면,300,g,780.0,18.0,3.1,170.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11726,생칼국수,2019,대판식품,생면,100,g,278.0,7.0,1.0,60.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11739,생칼국수,2019,미농식품,생면,167,g,465.0,11.0,1.7,101.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


## 바코드, 음식 조인

## 심플데이터

In [551]:
food_w_brcd = food.loc[:,['food_name','cmpny','food_cat']]

In [552]:
food_w_brcd.rename(columns={'food_name':'prdt'},inplace=True)

In [553]:
food_w_brcd

,prdt,cmpny,food_cat
0,구운인절미,해태가루비,스낵과자
1,코펜라스버터쿠키,매크로통상,쿠키
2,로아커웨하스바닐라,델리팜,웨이퍼
3,맛미강정,왕궁병과,한과류
4,찹쌀약과,성진식품,한과류
...,...,...,...
7699,<아내의식탁>짜.수.치,프레시지,떡볶이류
7700,<아내의식탁>바질베스토감바스,프레시지,어패류볶음
7701,<아내의식탁>밀푀유스키야키,프레시지,채소류찌개.전골
7702,<파빌리온>양갈비스테이크,프레시지,육류구이


In [554]:
brcd_df = pd.merge(brcd, food_w_brcd,  how='inner', on=['prdt', 'cmpny'])

In [555]:
brcd_df.head()

,prdt,brcd,cmpny,food_cat
0,면사랑구수한메밀국수,8802039211424,면사랑,건면
1,면사랑구수한메밀국수,8802039211455,면사랑,건면
2,면사랑구수한메밀국수,8802039211431,면사랑,건면
3,면사랑구수한메밀국수,8802039211417,면사랑,건면
4,퓨어네유기농오렌지,8809125060812,퓨어플러스,과/채주스


In [556]:
brcd_df[brcd_df['brcd'].duplicated()]

,prdt,brcd,cmpny,food_cat
10,커피맛캔디,8803318163069,신화당제과,사탕
48,쵸코파이,8801111370295,크라운제과,초콜릿가공품
49,쵸코파이,8801111370257,크라운제과,초콜릿가공품
50,쵸코파이,8801111370943,크라운제과,초콜릿가공품
51,쵸코파이,8801111370936,크라운제과,초콜릿가공품
...,...,...,...,...
33825,유기명일엽녹즙,8809231002447,풀무원건강생활,과/채주스
33826,생생우동,8801043003001,농심,생면
33827,사리곰탕큰사발,8801043805469,농심,유탕면
33829,키즈하이녹즙,8809231002621,풀무원건강생활,과/채주스


In [557]:
brcd_df[brcd_df['brcd']=='8801111370295']

,prdt,brcd,cmpny,food_cat
38,쵸코파이,8801111370295,크라운제과,초콜릿가공품
48,쵸코파이,8801111370295,크라운제과,초콜릿가공품


### 전체 중복 제거

In [558]:
brcd_df.drop_duplicates(inplace=True)

In [559]:
brcd_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28071 entries, 0 to 33831
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   prdt      28071 non-null  object
 1   brcd      28071 non-null  object
 2   cmpny     28071 non-null  object
 3   food_cat  28071 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


In [560]:
brcd_df[brcd_df['brcd'].duplicated()]

,prdt,brcd,cmpny,food_cat
191,화이트하임,8801111913195,크라운제과,과자
192,화이트하임,8801111913195,크라운제과,비스킷
194,화이트하임,8801111114431,크라운제과,과자
195,화이트하임,8801111114431,크라운제과,비스킷
197,화이트하임,8801111112741,크라운제과,과자
...,...,...,...,...
33825,유기명일엽녹즙,8809231002447,풀무원건강생활,과/채주스
33826,생생우동,8801043003001,농심,생면
33827,사리곰탕큰사발,8801043805469,농심,유탕면
33829,키즈하이녹즙,8809231002621,풀무원건강생활,과/채주스


In [561]:
brcd_df[brcd_df['brcd']=='8801043005500']

,prdt,brcd,cmpny,food_cat
79,새우깡,8801043005500,농심,스낵과자


In [563]:
brcd_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28071 entries, 0 to 33831
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   prdt      28071 non-null  object
 1   brcd      28071 non-null  object
 2   cmpny     28071 non-null  object
 3   food_cat  28071 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


In [565]:
brcd_df[brcd_df.duplicated(subset=['brcd'])]

,prdt,brcd,cmpny,food_cat
191,화이트하임,8801111913195,크라운제과,과자
192,화이트하임,8801111913195,크라운제과,비스킷
194,화이트하임,8801111114431,크라운제과,과자
195,화이트하임,8801111114431,크라운제과,비스킷
197,화이트하임,8801111112741,크라운제과,과자
...,...,...,...,...
33825,유기명일엽녹즙,8809231002447,풀무원건강생활,과/채주스
33826,생생우동,8801043003001,농심,생면
33827,사리곰탕큰사발,8801043805469,농심,유탕면
33829,키즈하이녹즙,8809231002621,풀무원건강생활,과/채주스


In [566]:
brcd_df[brcd_df['brcd']=='8809231002447']

,prdt,brcd,cmpny,food_cat
3064,유기명일엽녹즙,8809231002447,풀무원녹즙,과/채주스
33825,유기명일엽녹즙,8809231002447,풀무원건강생활,과/채주스


In [577]:
brcd_df[brcd_df.duplicated(subset=['brcd','prdt','food_cat','cmpny'])]

,prdt,brcd,cmpny,food_cat


In [578]:
brcd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228581 entries, 0 to 52474
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   prdt    228581 non-null  object
 1   brcd    228581 non-null  object
 2   cmpny   228581 non-null  object
dtypes: object(3)
memory usage: 7.0+ MB


### 바코드 중복 제거 ( 마지막꺼만 남기기 )

In [570]:
brcd_final = brcd_df.drop_duplicates(subset=['brcd'], keep='last')

### 파일로 저장

In [573]:
save_brcd_smpl = os.path.join(os.pardir,'data','brcd_final_simple.csv')

brcd_final.to_csv(save_brcd_smpl)

## 전체 데이터

In [130]:
food_bc = food.rename(columns={'food_name':'prdt'})

In [131]:
len(food_bc), len(brcd)

(70199, 228560)

In [132]:
brcd_total_df = pd.merge(brcd, food_bc,  how='inner', on=['prdt', 'cmpny'])

In [133]:
brcd_total_df.head()

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
0,면사랑구수한메밀국수,8802039211424,면사랑,2019,건면,100,g,335.0,10.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,면사랑구수한메밀국수,8802039211455,면사랑,2019,건면,100,g,335.0,10.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,면사랑구수한메밀국수,8802039211431,면사랑,2019,건면,100,g,335.0,10.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,면사랑구수한메밀국수,8802039211417,면사랑,2019,건면,100,g,335.0,10.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,퓨어네유기농오렌지,8809125060812,퓨어플러스,2019,과/채주스,245,mL,96.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
brcd_total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33830 entries, 0 to 33829
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   prdt                       33830 non-null  object 
 1   brcd                       33830 non-null  object 
 2   cmpny                      33830 non-null  object 
 3   year                       33830 non-null  object 
 4   food_cat                   33830 non-null  object 
 5   serving_amount             33830 non-null  object 
 6   serving_unit               33676 non-null  object 
 7   Calories(kcal)             33830 non-null  float64
 8   Protein(g)                 33830 non-null  float64
 9   Fat(g)                     33830 non-null  float64
 10  Carbohydrate(g)            33830 non-null  float64
 11  Dietary_fiber(g)           33830 non-null  float64
 12  Calcium(mg)                33830 non-null  float64
 13  Iron(mg)                   33830 non-null  flo

### 전체 중복 제거

In [135]:
brcd_total_df.drop_duplicates(inplace=True, ignore_index=True)

In [136]:
brcd_total_df[brcd_total_df['brcd']=='8801111370295']

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
37,쵸코파이,8801111370295,크라운제과,2019,초콜릿가공품,35,g,165.0,2.0,7.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
brcd_total_df.head()

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
0,면사랑구수한메밀국수,8802039211424,면사랑,2019,건면,100,g,335.0,10.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,면사랑구수한메밀국수,8802039211455,면사랑,2019,건면,100,g,335.0,10.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,면사랑구수한메밀국수,8802039211431,면사랑,2019,건면,100,g,335.0,10.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,면사랑구수한메밀국수,8802039211417,면사랑,2019,건면,100,g,335.0,10.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,퓨어네유기농오렌지,8809125060812,퓨어플러스,2019,과/채주스,245,mL,96.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
brcd_total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33378 entries, 0 to 33377
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   prdt                       33378 non-null  object 
 1   brcd                       33378 non-null  object 
 2   cmpny                      33378 non-null  object 
 3   year                       33378 non-null  object 
 4   food_cat                   33378 non-null  object 
 5   serving_amount             33378 non-null  object 
 6   serving_unit               33224 non-null  object 
 7   Calories(kcal)             33378 non-null  float64
 8   Protein(g)                 33378 non-null  float64
 9   Fat(g)                     33378 non-null  float64
 10  Carbohydrate(g)            33378 non-null  float64
 11  Dietary_fiber(g)           33378 non-null  float64
 12  Calcium(mg)                33378 non-null  float64
 13  Iron(mg)                   33378 non-null  flo

In [139]:
brcd_total_df[brcd_total_df['brcd'].duplicated()]

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
71,새우깡,8801043026468,농심,2020,스낵과자,30,g,150.00,2.00,8.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
72,새우깡,8801043026468,농심,2021,스낵과자,90,g,465.00,6.00,25.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
74,새우깡,8801043005500,농심,2020,스낵과자,30,g,150.00,2.00,8.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
75,새우깡,8801043005500,농심,2021,스낵과자,90,g,465.00,6.00,25.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
77,새우깡,8801043102780,농심,2020,스낵과자,30,g,150.00,2.00,8.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33371,유기명일엽녹즙,8809231002447,풀무원건강생활,2019,과/채주스,130,mL,20.00,0.70,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
33372,생생우동,8801043003001,농심,2017,생면,253,g,405.00,0.00,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
33373,사리곰탕큰사발,8801043805469,농심,2017,유탕면,80,g,285.41,4.61,9.8,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
33375,키즈하이녹즙,8809231002621,풀무원건강생활,2019,과/채주스,130,mL,55.00,0.00,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


### 연도 순으로 정렬

In [142]:
brcd_total_df.sort_values(by=['year'],ignore_index=True, inplace=True)

In [143]:
brcd_total_df[brcd_total_df['brcd']=='8809424550069']

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
11684,플레인베이글,8809424550069,고메베이글,2020,빵류,110,g,276.0,11.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
18512,플레인베이글A,8809424550069,고메베이글,2021,빵류,110,g,275.0,11.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
brcd_total_df.drop_duplicates(subset=['brcd'], keep='last', inplace=True, ignore_index=True)

In [145]:
brcd_total_df[brcd_total_df['brcd']=='8809424550069']

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
10927,플레인베이글A,8809424550069,고메베이글,2021,빵류,110,g,275.0,11.0,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
brcd_total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25267 entries, 0 to 25266
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   prdt                       25267 non-null  object 
 1   brcd                       25267 non-null  object 
 2   cmpny                      25267 non-null  object 
 3   year                       25267 non-null  object 
 4   food_cat                   25267 non-null  object 
 5   serving_amount             25267 non-null  object 
 6   serving_unit               25167 non-null  object 
 7   Calories(kcal)             25267 non-null  float64
 8   Protein(g)                 25267 non-null  float64
 9   Fat(g)                     25267 non-null  float64
 10  Carbohydrate(g)            25267 non-null  float64
 11  Dietary_fiber(g)           25267 non-null  float64
 12  Calcium(mg)                25267 non-null  float64
 13  Iron(mg)                   25267 non-null  flo

In [147]:
brcd_total_df[brcd_total_df['prdt'].str.contains('메로나',na=False) & (brcd_total_df['cmpny']=='빙그레')]

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
5186,수출용메로나,769828120801,빙그레,2020,아이스밀크,70,mL,120.0,1.0,4.50,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5289,수출용메로나,769828120825,빙그레,2020,아이스밀크,70,mL,120.0,1.0,4.50,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
5735,메로나딸기,8801104900126,빙그레,2020,아이스밀크,70,mL,110.0,1.0,3.50,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6749,메로나바나나,8801104900089,빙그레,2020,아이스크림,75,mL,150.0,2.5,3.88,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6750,메로나바나나,8801104900072,빙그레,2020,아이스크림,75,mL,150.0,2.5,3.88,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6751,메로나바나나,8801104900140,빙그레,2020,아이스크림,75,mL,150.0,2.5,3.88,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
17459,메로나피나콜라다,8801104671040,빙그레,2021,샤베트,75,g,105.0,1.3,1.50,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
21015,메로나,8801104123471,빙그레,2021,아이스밀크,75,g,120.0,1.0,5.00,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
21063,메로나,8801104143011,빙그레,2021,아이스밀크,75,g,120.0,1.0,5.00,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
21083,메로나,8801104122382,빙그레,2021,아이스밀크,75,g,120.0,1.0,5.00,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


### 파일로 저장

In [158]:
# 매운새우깡
brcd_total_df[brcd_total_df['prdt'] == '매운새우깡']

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
11030,매운새우깡,880104310282,농심,2021,스낵과자,90,g,450.0,6.0,20.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11031,매운새우깡,8801043019354,농심,2021,스낵과자,90,g,450.0,6.0,20.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11032,매운새우깡,8801043004183,농심,2021,스낵과자,90,g,450.0,6.0,20.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11079,매운새우깡,8801043102827,농심,2021,스낵과자,90,g,450.0,6.0,20.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11081,매운새우깡,8801043026321,농심,2021,스낵과자,90,g,450.0,6.0,20.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11082,매운새우깡,8801043102018,농심,2021,스낵과자,90,g,450.0,6.0,20.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11126,매운새우깡,8801043806626,농심,2021,스낵과자,90,g,450.0,6.0,20.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11143,매운새우깡,8801043026369,농심,2021,스낵과자,90,g,450.0,6.0,20.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
# 칠성사이다
brcd_total_df[brcd_total_df['prdt'] == '코카콜라제로']

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
12850,코카콜라제로,8801094083854,코카콜라음료,2021,탄산음료,500,g,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
12852,코카콜라제로,8801094083038,코카콜라음료,2021,탄산음료,500,g,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
12854,코카콜라제로,8801094089108,코카콜라음료,2021,탄산음료,500,g,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
12857,코카콜라제로,8801094083205,코카콜라음료,2021,탄산음료,500,g,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
13195,코카콜라제로,8801094083007,코카콜라음료,2021,탄산음료,500,g,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
13209,코카콜라제로,8801094083830,코카콜라음료,2021,탄산음료,500,g,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
13231,코카콜라제로,8801094083885,코카콜라음료,2021,탄산음료,500,g,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
13233,코카콜라제로,8801094082802,코카콜라음료,2021,탄산음료,500,g,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
13253,코카콜라제로,8801094082703,코카콜라음료,2021,탄산음료,500,g,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
13255,코카콜라제로,8801094082604,코카콜라음료,2021,탄산음료,500,g,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
# 비비빅
brcd_total_df[brcd_total_df['prdt'] == '비비빅']

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
11085,비비빅,769828120115,빙그레,2021,샤베트,70,g,135.0,2.3,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11086,비비빅,769828160210,빙그레,2021,샤베트,70,g,135.0,2.3,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11087,비비빅,8801104185721,빙그레,2021,샤베트,70,g,135.0,2.3,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11090,비비빅,8801104123242,빙그레,2021,샤베트,70,g,135.0,2.3,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
11097,비비빅,8801104123181,빙그레,2021,샤베트,70,g,135.0,2.3,1.3,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
brcd_test_df = pd.concat(
    [
        brcd_total_df[brcd_total_df['prdt'] == '매운새우깡'],
        brcd_total_df[brcd_total_df['prdt'] == '코카콜라제로'],
        brcd_total_df[brcd_total_df['prdt'] == '비비빅']
    ]
)

In [184]:
brcd_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 11030 to 11097
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   prdt                       26 non-null     object 
 1   brcd                       26 non-null     object 
 2   cmpny                      26 non-null     object 
 3   year                       26 non-null     object 
 4   food_cat                   26 non-null     object 
 5   serving_amount             26 non-null     object 
 6   serving_unit               26 non-null     object 
 7   Calories(kcal)             26 non-null     float64
 8   Protein(g)                 26 non-null     float64
 9   Fat(g)                     26 non-null     float64
 10  Carbohydrate(g)            26 non-null     float64
 11  Dietary_fiber(g)           26 non-null     float64
 12  Calcium(mg)                26 non-null     float64
 13  Iron(mg)                   26 non-null     fl

In [258]:
save_brcd = os.path.join(os.pardir,'data','brcd_final.csv')

brcd_test_df.to_csv(save_brcd, index=False)

# DB테스트

In [655]:
data_df = pd.read_csv(save_brcd, dtype=str)

In [656]:
data_df.head()

,prdt,brcd,cmpny,year,food_cat,serving_amount,serving_unit,Calories(kcal),Protein(g),Fat(g),...,Histidine(g),Linoleic_acid(g),Alpha-linolenic_acid(g),EPA+DHA(mg),Isoleucine(g),Lysine(g),Methionine(g),Phenylalanine+Tyrosine(g),Threonine(g),Valine(g)
0,허니통통,8801019606830,해태제과식품,2016,스낵과자,30,g,160.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,허니통통,8801019606915,해태제과식품,2016,스낵과자,30,g,160.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,허니통통,8801019607448,해태제과식품,2016,스낵과자,30,g,160.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,꿀먹은아이홍삼,8809289107972,고철남홍삼,2016,인삼/홍삼음료,20,g,5.96,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,꿀먹은아이홍삼,8809289107651,고철남홍삼,2016,인삼/홍삼음료,20,g,5.96,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [677]:
test_df = pd.DataFrame(columns=['PK','SK','food_name','cmpny','food_cat','qty','nutrients'])

In [678]:
# SK
test_df['SK'] = 'BRCD#' + data_df['brcd']

In [679]:
# PK
test_df['PK'] = 'BRCD#brcd'

In [680]:
test_df.head()

,PK,SK,food_name,cmpny,food_cat,qty,nutrients
0,BRCD#brcd,BRCD#8801019606830,NaN,NaN,NaN,NaN,NaN
1,BRCD#brcd,BRCD#8801019606915,NaN,NaN,NaN,NaN,NaN
2,BRCD#brcd,BRCD#8801019607448,NaN,NaN,NaN,NaN,NaN
3,BRCD#brcd,BRCD#8809289107972,NaN,NaN,NaN,NaN,NaN
4,BRCD#brcd,BRCD#8809289107651,NaN,NaN,NaN,NaN,NaN


In [681]:
test_df['cmpny'] = data_df['cmpny']

In [682]:
test_df['food_cat'] = data_df['food_cat']

In [683]:
brcd_df['food_name'] = data_df['prdt']

In [684]:
test_df['qty'] = data_df[['serving_amount','serving_unit']].apply(dict, axis=1)

In [685]:
to_del = ['prdt', 'brcd', 'cmpny', 'year', 'food_cat', 'serving_amount', 'serving_unit',]

In [686]:
data_col = data_df.columns

In [687]:
nutr_col = [col for col in data_col if col not in to_del]

In [688]:
drop0 = lambda row: row.dropna().to_dict()

In [689]:
test_df['nutrients'] = data_df[nutr_col].apply(drop0, axis=1)

In [690]:
test_df.head()

,PK,SK,food_name,cmpny,food_cat,qty,nutrients
0,BRCD#brcd,BRCD#8801019606830,NaN,해태제과식품,스낵과자,"{'serving_amount': '30', 'serving_unit': 'g'}","{'Calories(kcal)': '160.0', 'Protein(g)': '0.0..."
1,BRCD#brcd,BRCD#8801019606915,NaN,해태제과식품,스낵과자,"{'serving_amount': '30', 'serving_unit': 'g'}","{'Calories(kcal)': '160.0', 'Protein(g)': '0.0..."
2,BRCD#brcd,BRCD#8801019607448,NaN,해태제과식품,스낵과자,"{'serving_amount': '30', 'serving_unit': 'g'}","{'Calories(kcal)': '160.0', 'Protein(g)': '0.0..."
3,BRCD#brcd,BRCD#8809289107972,NaN,고철남홍삼,인삼/홍삼음료,"{'serving_amount': '20', 'serving_unit': 'g'}","{'Calories(kcal)': '5.96', 'Protein(g)': '0.0'..."
4,BRCD#brcd,BRCD#8809289107651,NaN,고철남홍삼,인삼/홍삼음료,"{'serving_amount': '20', 'serving_unit': 'g'}","{'Calories(kcal)': '5.96', 'Protein(g)': '0.0'..."
